<a href="https://colab.research.google.com/github/pratik1208/HealthProPred/blob/main/Target_Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
df= pd.read_csv("/content/drive/MyDrive/Doceree-HCP_Train.csv",encoding='latin-1')

In [ ]:
df.columns

In [ ]:
df['KEYWORDS'] =df['KEYWORDS'].str.lower()

In [ ]:
# data = df[['DEVICETYPE','USERCITY','USERZIPCODE','USERAGENT','PLATFORMTYPE','URL','IS_HCP']].fillna(0)
# columns = data.columns

In [ ]:
data = df.copy()

In [ ]:
data

In [ ]:
data.drop(["TAXONOMY","KEYWORDS","CHANNELTYPE","PLATFORMTYPE","USERPLATFORMUID","BIDREQUESTIP","ID","PLATFORM_ID","DEVICETYPE"], axis=1, inplace=True)
data = data.fillna(0)

In [ ]:
columns = data.columns

In [ ]:
test = data.sample(20000,random_state=42)
train = data[~data.index.isin(test.index)]

#**Target Encoding**

In [ ]:
!pip install category_encoders

In [ ]:
train_x = train.iloc[:,:-1]
train_y = train.iloc[:,-1]
test_x = test.iloc[:,:-1]
test_y = test.iloc[:,-1]

In [ ]:
import category_encoders as ce


#Create target encoding object
encoder=ce.TargetEncoder(cols=train_x.columns, smoothing=0.2)


In [ ]:
encoder

In [ ]:
train_x_encoded = encoder.fit_transform(train_x, train_y)
test_x_encoded =encoder.transform(test_x, test_y)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()

In [ ]:
dt.fit(train_x_encoded, train_y)
y_pred = dt.predict(test_x_encoded)
y_pred_train = dt.predict(train_x_encoded)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
print(f1_score(y_pred,test_y))
print(accuracy_score(y_pred,test_y))

In [ ]:
importance = dt.feature_importances_
feature_names = train_x.columns

In [ ]:
importance


In [ ]:
import matplotlib.pyplot as plt
plt.barh(range(len(importance)), importance, align = 'center')
plt.xlabel("Feature Importance")
plt.ylabel("Feature")
plt.yticks(range(len(importance)), feature_names)
for index, value in enumerate(importance):
    plt.text(value, index, f'{value:.4f}')
plt.show()

**Count Vectorizer**

In [ ]:
train_x = train_x.join(df.KEYWORDS, how= 'left').copy()
test_x = test_x.join(df.KEYWORDS, how= 'left').copy()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=514)


In [ ]:
vectorizer

In [ ]:
train_x_sparse_matrix = vectorizer.fit_transform(train_x.KEYWORDS)
test_x_sparse_matrix = vectorizer.transform(test_x.KEYWORDS)

In [ ]:
feature_names = vectorizer.get_feature_names_out()


In [ ]:
test_x_sparse_matrix

In [ ]:
df_train = pd.DataFrame(train_x_sparse_matrix.toarray(), columns=feature_names,index = train_x.index)
df_test = pd.DataFrame(test_x_sparse_matrix.toarray(), columns=feature_names,index = test_x.index)



In [ ]:
final_train = train_x_encoded.join(df_train, how='left')
final_test = test_x_encoded.join(df_test, how= 'left')

**Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(final_train, train_y)

In [ ]:
y_pred = dt.predict(final_test)
y_pred_train = dt.predict(final_train)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
print("Train F1 score is",f1_score(train_y, y_pred_train, average='macro'))
print("Test F1 score is",f1_score(test_y, y_pred, average = 'macro'))
print("Train Accuracy is",accuracy_score(train_y, y_pred_train))
print("Test Accuracy is",accuracy_score(test_y, y_pred))
print("Train Precision is",precision_score(train_y, y_pred_train))
print("Test Precision is",precision_score(test_y, y_pred))
print("Train Recall is",recall_score(train_y, y_pred_train))
print("Test Recall is",recall_score(test_y, y_pred))

In [ ]:
from tabulate import tabulate
table_data = [
    ["Metric", "Train", "Test"],
    ["F1 Score", f1_score(train_y, y_pred_train, average='macro'), f1_score(test_y, y_pred, average = 'macro')],
    ["Accuracy", accuracy_score(train_y, y_pred_train), accuracy_score(test_y, y_pred)],
    ["Precision", precision_score(train_y, y_pred_train), precision_score(test_y, y_pred)],
    ["Recall", recall_score(train_y, y_pred_train), recall_score(test_y, y_pred)],
]

# Print the table
print(tabulate(table_data, headers="firstrow", tablefmt="grid"))

###**Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100,max_features=3, random_state=42)



In [ ]:
rf.fit(final_train,train_y)

In [ ]:
y_pred_rf = rf.predict(final_test)
y_pred_train_rf = rf.predict(final_train)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
print("Train F1 score is",f1_score(train_y, y_pred_train_rf, average='macro'))
print("Test F1 score is",f1_score(test_y, y_pred_rf, average = 'macro'))
print("Train F1 score is",f1_score(train_y, y_pred_train_rf))
print("Test F1 score is",f1_score(test_y, y_pred_rf))
print("Train Accuracy is",accuracy_score(train_y, y_pred_train_rf))
print("Test Accuracy is",accuracy_score(test_y, y_pred_rf))
print("Train Precision is",precision_score(train_y, y_pred_train_rf))
print("Test Precision is",precision_score(test_y, y_pred_rf))
print("Train Recall is",recall_score(train_y, y_pred_train_rf))
print("Test Recall is",recall_score(test_y, y_pred_rf))


In [ ]:
from tabulate import tabulate
table_data = [
    ["Metric", "Train", "Test"],
    ["F1 Score", f1_score(train_y, y_pred_train_rf, average='macro'), f1_score(test_y, y_pred_rf, average = 'macro')],
    ["Accuracy", accuracy_score(train_y, y_pred_train_rf), accuracy_score(test_y, y_pred_rf)],
    ["Precision", precision_score(train_y, y_pred_train_rf), precision_score(test_y, y_pred_rf)],
    ["Recall",recall_score(train_y, y_pred_train_rf), recall_score(test_y, y_pred_rf)],
]

# Print the table
print(tabulate(table_data, headers="firstrow", tablefmt="grid"))

#**Gradient Boosing**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
gb.fit(final_train,train_y)
y_pred_gb = gb.predict(final_test)
y_pred_train_gb = gb.predict(final_train)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
print("Train F1 score is",f1_score(train_y, y_pred_train_gb, average='macro'))
print("Test F1 score is",f1_score(test_y, y_pred_gb, average = 'macro'))
print("Train F1 score is",f1_score(train_y, y_pred_train_gb))
print("Test F1 score is",f1_score(test_y, y_pred_gb))
print("Train Accuracy is",accuracy_score(train_y, y_pred_train_gb))
print("Test Accuracy is",accuracy_score(test_y, y_pred_gb))
print("Train Precision is",precision_score(train_y, y_pred_train_gb))
print("Test Precision is",precision_score(test_y, y_pred_gb))
print("Train Recall is",recall_score(train_y, y_pred_train_gb))
print("Test Recall is",recall_score(test_y, y_pred_gb))


#**XGBoost**

In [ ]:
import xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix(final_train, label=train_y)
dtest = xgb.DMatrix(final_test, label=test_y)

In [ ]:
params = {
    'objective': 'multi:softmax',      # Classification task with multiple classes
    'eval_metric': 'merror',            # Evaluation metric for multi-class classification
    'num_class': len(set(train_y)),     # Number of classes in the target variable
    'max_depth': 3,                     # Maximum depth of the tree
    'learning_rate': 0.1,               # Learning rate
    'subsample': 0.8,                   # Fraction of training data to use in each boosting round
    'colsample_bytree': 0.8,            # Fraction of features to use in each boosting round
    'n_estimators': 100                 # Number of boosting rounds
}

In [ ]:
model = xgb.train(params, dtrain, num_boost_round=100)
y_pred = model.predict(dtest)

In [ ]:
y_pred_xgb = model.predict(final_test)
y_pred_train_xgb = model.predict(final_train)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
print("Train F1 score is",f1_score(train_y, y_pred_train_xgb, average='macro'))
print("Test F1 score is",f1_score(test_y, y_pred_xgb, average = 'macro'))
print("Train F1 score is",f1_score(train_y, y_pred_train_xgb))
print("Test F1 score is",f1_score(test_y, y_pred_xgb))
print("Train Accuracy is",accuracy_score(train_y, y_pred_train_xgb))
print("Test Accuracy is",accuracy_score(test_y, y_pred_xgb))
print("Train Precision is",precision_score(train_y, y_pred_train_xgb))
print("Test Precision is",precision_score(test_y, y_pred_xgb))
print("Train Recall is",recall_score(train_y, y_pred_train_xgb))
print("Test Recall is",recall_score(test_y, y_pred_xgb))
